## Домашнее задание №3

В данном задании вам необходимо реализовать функции ошибки для линейной регрессии и их производные по параметрам, __не используя автоматические дифференцирование.__ Все методы должны быть реализованы только с использованием библиотеки `numpy`. 

Ваша основная задача: вывести формулы для производных __MSE, MAE, L1 и L2 регуляризационных членов__ в _векторном случае_ (т.е. когда и объект $\mathbf{x}_i$, и целевое значение $\mathbf{y}_i$ являются векторами.


Для работы вновь обратимся к [Boston housing prices dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html). Он был предобработан для вашего удобства и будет загружен ниже.

In [1]:
# %pip install numpy pandas seaborn scikit-learn matplotlib scipy

In [2]:
"""
If you are using Google Colab, uncomment the next line to download `boston_subset.json`
"""

# !wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_mipt/homeworks/assignment03_derivatives/boston_subset.json

'\nIf you are using Google Colab, uncomment the next line to download `boston_subset.json`\n'

In [3]:
import json

# Run some setup code for this notebook.
import random

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge

In [4]:
with open("boston_subset.json") as iofile:
    dataset = json.load(iofile)
feature_matrix = np.array(dataset["data"])
targets = np.array(dataset["target"])

## Имплементация функций потерь и методов регуляризации.
Для того, чтобы решить задание, вам необходимо реализовать все методы в файле `loss_and_derivatives.py`.
__Внимание, в данном задании не требуется использовать свободный член (bias term)__, т.е. линейная модель примет простой вид
$$
\hat{\mathbf{y}} = XW
$$
Единичный столбец также не добавляется к матрице $X$.

Реализуйте методы для MSE, MAE, L1 и L2 регуляризации, а также вычисления их производных (опциональное задание) по параметрам линейной модели.

__Для вашего удобства данные уже предобработаны, и использование линейной модели без свободного члена не является ошибкой. В данном задании он не должен быть использован.__

In [51]:
class LossAndDerivatives:
    @staticmethod
    def mse(X, Y, w):
        """
        X : numpy array of shape (`n_observations`, `n_features`)
        Y : numpy array of shape (`n_observations`, `target_dimentionality`) or (`n_observations`,)
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return : float
            single number with MSE value of linear model (X.dot(w)) with no bias term
            on the selected dataset.

        Comment: If Y is two-dimentional, average the error over both dimentions.
        """

        return np.mean((X.dot(w) - Y) ** 2)

    @staticmethod
    def mae(X, Y, w):
        """
        X : numpy array of shape (`n_observations`, `n_features`)
        Y : numpy array of shape (`n_observations`, `target_dimentionality`) or (`n_observations`,)
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return: float
            single number with MAE value of linear model (X.dot(w)) with no bias term
            on the selected dataset.

        Comment: If Y is two-dimentional, average the error over both dimentions.
        """

        # YOUR CODE HERE
        return np.sum(np.abs(X.dot(w) - Y))

    @staticmethod
    def l2_reg(w):
        """
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return: float
            single number with sum of squared elements of the weight matrix ( \sum_{ij} w_{ij}^2 )

        Computes the L2 regularization term for the weight matrix w.
        """

        # YOUR CODE HERE
        return np.einsum('ij,ij->ij', w, w)

    @staticmethod
    def l1_reg(w):
        """
        w : numpy array of shape (`n_features`, `target_dimentionality`)

        Return : float
            single number with sum of the absolute values of the weight matrix ( \sum_{ij} |w_{ij}| )

        Computes the L1 regularization term for the weight matrix w.
        """

        # YOUR CODE HERE
        return np.einsum('ij,ij->ij', abs(w), abs(w))

    @staticmethod
    def no_reg(w):
        """
        Simply ignores the regularization
        """
        return 0.0

    @staticmethod
    def mse_derivative(X, Y, w):
        """
        X : numpy array of shape (`n_observations`, `n_features`)
        Y : numpy array of shape (`n_observations`, `target_dimentionality`) or (`n_observations`,)
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return : numpy array of same shape as `w`

        Computes the MSE derivative for linear regression (X.dot(w)) with no bias term
        w.r.t. w weight matrix.

        Please mention, that in case `target_dimentionality` > 1 the error is averaged along this
        dimension as well, so you need to consider that fact in derivative implementation.
        """

        # YOUR CODE HERE
        return 2 / X.shape[1] * np.dot(X.T, (np.dot(X, w) - Y)) 

    @staticmethod
    def mae_derivative(X, Y, w):
        """
        X : numpy array of shape (`n_observations`, `n_features`)
        Y : numpy array of shape (`n_observations`, `target_dimentionality`) or (`n_observations`,)
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return : numpy array of same shape as `w`

        Computes the MAE derivative for linear regression (X.dot(w)) with no bias term
        w.r.t. w weight matrix.

        Please mention, that in case `target_dimentionality` > 1 the error is averaged along this
        dimension as well, so you need to consider that fact in derivative implementation.
        """

        # YOUR CODE HERE
        if len(Y.shape) > 1:    
            return  X.T.dot(np.sign(X.dot(w) - Y)) / (Y.shape[0] * Y.shape[1])
        else: return  X.T.dot(np.sign(X.dot(w) - Y)) / (Y.shape[0])

    @staticmethod
    def l2_reg_derivative(w):
        """
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return : numpy array of same shape as `w`

        Computes the L2 regularization term derivative w.r.t. the weight matrix w.
        """

        # YOUR CODE HERE
        return np.zeros_like(w)

    @staticmethod
    def l1_reg_derivative(w):
        """
        Y : numpy array of shape (`n_observations`, `target_dimentionality`) or (`n_observations`,)
        w : numpy array of shape (`n_features`, `target_dimentionality`) or (`n_features`,)

        Return : numpy array of same shape as `w`

        Computes the L1 regularization term derivative w.r.t. the weight matrix w.
        """

        # YOUR CODE HERE
        return np.zeros_like(w)

    @staticmethod
    def no_reg_derivative(w):
        """
        Simply ignores the derivative
        """
        return np.zeros_like(w)

<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:51: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:51: SyntaxWarning: invalid escape sequence '\s'
C:\Users\malne\AppData\Local\Temp\ipykernel_1112\4175072765.py:37: SyntaxWarning: invalid escape sequence '\s'
  """
C:\Users\malne\AppData\Local\Temp\ipykernel_1112\4175072765.py:51: SyntaxWarning: invalid escape sequence '\s'
  """


In [52]:
x = np.array([2, 4, 3])
z = np.array([-1, 3, 4])
np.einsum('i,j->ij', x, z)

array([[-2,  6,  8],
       [-4, 12, 16],
       [-3,  9, 12]])

Обращаем ваше внимание, требуется реализовать решение в векторном виде (т.е. для каждого объекта предсказание $\hat{\mathbf{y}}$ является вектором с размерностью $\geq 1$. При подсчете ошибки она усредняется как по объектам, так и по размерности __y__.

Например, для вектора ошибок на одном объекте $[1., 1., 1., 1.]$ значение функции ошибки будет равно $\frac{1}{4}(1. + 1. + 1. + 1.)$ 

Для вашего удобства метод `.mse` уже реализован и вы можете обращаться к нему за примером.

Для проверки своего кода вам доступно несколько assert'ов:

In [53]:
w = np.array([1.0, 1.0])
x_n, y_n = feature_matrix, targets

# Repeating data to make everything multi-dimentional
w = np.vstack([w[None, :] + 0.27, w[None, :] + 0.22, w[None, :] + 0.45, w[None, :] + 0.1]).T
y_n = np.hstack([y_n[:, None], 2 * y_n[:, None], 3 * y_n[:, None], 4 * y_n[:, None]])

In [54]:
reference_mse_derivative = np.array(
    [
        [7.32890068, 12.88731311, 18.82128365, 23.97731238],
        [9.55674399, 17.05397661, 24.98807528, 32.01723714],
    ]
)
reference_l2_reg_derivative = np.array([[2.54, 2.44, 2.9, 2.2], [2.54, 2.44, 2.9, 2.2]])

assert np.allclose(
    reference_mse_derivative, LossAndDerivatives.mse_derivative(x_n, y_n, w), rtol=1e-3
), "Something wrong with MSE derivative"

assert np.allclose(
    reference_l2_reg_derivative, LossAndDerivatives.l2_reg_derivative(w), rtol=1e-3
), "Something wrong with L2 reg derivative"

print(
    "MSE derivative:\n{} \n\nL2 reg derivative:\n{}".format(
        LossAndDerivatives.mse_derivative(x_n, y_n, w),
        LossAndDerivatives.l2_reg_derivative(w),
    )
)

AssertionError: Something wrong with MSE derivative

In [55]:
reference_mae_derivative = np.array(
    [
        [0.19708867, 0.19621798, 0.19621798, 0.19572906],
        [0.25574138, 0.25524507, 0.25524507, 0.25406404],
    ]
)
reference_l1_reg_derivative = np.array([[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0]])

assert np.allclose(
    reference_mae_derivative, LossAndDerivatives.mae_derivative(x_n, y_n, w), rtol=1e-3
), "Something wrong with MAE derivative"

assert np.allclose(
    reference_l1_reg_derivative, LossAndDerivatives.l1_reg_derivative(w), rtol=1e-3
), "Something wrong with L1 reg derivative"

print(
    "MAE derivative:\n{} \n\nL1 reg derivative:\n{}".format(
        LossAndDerivatives.mae_derivative(x_n, y_n, w),
        LossAndDerivatives.l1_reg_derivative(w),
    )
)

AssertionError: Something wrong with L1 reg derivative

### Градиентный спуск для решения реальной задачи
Следующая функция позволяет найти оптимальные значения параметров с помощью градиентного спуска:

In [ ]:
def get_w_by_grad(X, Y, w_0, loss_mode="mse", reg_mode=None, lr=0.05, n_steps=100, reg_coeff=0.05):
    if loss_mode == "mse":
        loss_function = LossAndDerivatives.mse
        loss_derivative = LossAndDerivatives.mse_derivative
    elif loss_mode == "mae":
        loss_function = LossAndDerivatives.mae
        loss_derivative = LossAndDerivatives.mae_derivative
    else:
        raise ValueError("Unknown loss function. Available loss functions: `mse`, `mae`")

    if reg_mode is None:
        reg_function = LossAndDerivatives.no_reg
        reg_derivative = LossAndDerivatives.no_reg_derivative  # lambda w: np.zeros_like(w)
    elif reg_mode == "l2":
        reg_function = LossAndDerivatives.l2_reg
        reg_derivative = LossAndDerivatives.l2_reg_derivative
    elif reg_mode == "l1":
        reg_function = LossAndDerivatives.l1_reg
        reg_derivative = LossAndDerivatives.l1_reg_derivative
    else:
        raise ValueError("Unknown regularization mode. Available modes: `l1`, `l2`, None")

    w = w_0.copy()

    for i in range(n_steps):
        empirical_risk = loss_function(X, Y, w) + reg_coeff * reg_function(w)
        gradient = loss_derivative(X, Y, w) + reg_coeff * reg_derivative(w)
        gradient_norm = np.linalg.norm(gradient)
        if gradient_norm > 5.0:
            gradient = gradient / gradient_norm * 5.0
        w -= lr * gradient

        if i % 25 == 0:
            print("Step={}, loss={},\ngradient values={}\n".format(i, empirical_risk, gradient))
    return w

Рассмотрим простой пример:

In [ ]:
# Initial weight matrix
w = np.ones((2, 1), dtype=float)
y_n = targets[:, None]

In [ ]:
w_grad = get_w_by_grad(x_n, y_n, w, loss_mode="mse", reg_mode="l2", n_steps=250)

### Сравнение с `sklearn`
Сравним реализованную модель с версией из `sklearn`:

In [ ]:
lr = Ridge(alpha=0.05)
lr.fit(x_n, y_n)
print(
    "sklearn linear regression implementation delivers MSE = {}".format(
        np.mean((lr.predict(x_n) - y_n) ** 2)
    )
)

In [ ]:
plt.scatter(x_n[:, -1], y_n[:, -1])
plt.scatter(
    x_n[:, -1],
    x_n.dot(w_grad)[:, -1],
    color="orange",
    label="Handwritten linear regression",
    linewidth=5,
)
plt.scatter(x_n[:, -1], lr.predict(x_n), color="cyan", label="sklearn Ridge")
plt.legend()
plt.show()

Если в полученных решениях есть небольшие различия, это не страшно. Модель основанная на вашей реализации не использует свободный член (он равен $0$), в то время версия из `sklearn` настраивает и его.

### Сдача задания
Сдайте в чекер реализованный класс `LossAndDerivatives`. Для этого можете скопировать всю ячейку с кодом (в том числе и импортирование `numpy`) в файл `derivatives.py`.

На этом задание завершено. Поздравляем!